In [1]:
from model_builder import model_builder
from data_gen_ext import simulate_private_equity_cashflows_ext
import numpy as np

In [2]:
cashflows = simulate_private_equity_cashflows_ext()

In [3]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

cashflows

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,date,Cashflow
0,0,2002,Growth,Asia,Good,0,2002-03-31,-0.229863
1,0,2002,Growth,Asia,Good,1,2002-06-30,0.000000
2,0,2002,Growth,Asia,Good,2,2002-09-30,-0.134382
3,0,2002,Growth,Asia,Good,3,2002-12-31,0.000000
4,0,2002,Growth,Asia,Good,4,2003-03-31,-0.117993
...,...,...,...,...,...,...,...,...
118740,2399,2002,Buyout,Asia,Good,50,2014-09-30,0.034115
118741,2399,2002,Buyout,Asia,Good,51,2014-12-31,0.000000
118742,2399,2002,Buyout,Asia,Good,52,2015-03-31,0.000000
118743,2399,2002,Buyout,Asia,Good,53,2015-06-30,0.088846


In [4]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [5]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, port_cfs)
model

({'Portfolio MOIC': 2.1482651727900914,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 0.013055961746447853,
  'Strategy Alpha MOIC': -0.012330854345893094,
  'Geography Alpha MOIC': -0.01908166147731105,
  'Sizing Alpha MOIC': -0.01012836306405962,
  'Residual Alpha MOIC': 0.09970185005520893},
 {'Portfolio IRR': 0.13409949856468345,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': 0.003919674376440335,
  'Strategy Alpha IRR': -0.0018335582581840842,
  'Geography Alpha IRR': -0.0026395418891384526,
  'Sizing Alpha IRR': 0.00048109334161342,
  'Residual Alpha IRR': 0.007959411616698372})

### Monte-Carlo simulation

In [6]:
market_model = model_builder(cashflows, cashflows)
market_model

({'Portfolio MOIC': 2.0770482398756984,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 4.440892098500626e-16,
  'Strategy Alpha MOIC': 4.440892098500626e-16,
  'Geography Alpha MOIC': 4.440892098500626e-16,
  'Sizing Alpha MOIC': 0.0,
  'Residual Alpha MOIC': -1.3322676295501878e-15},
 {'Portfolio IRR': 0.12621241937725386,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': -5.551115123125783e-17,
  'Strategy Alpha IRR': -5.551115123125783e-17,
  'Geography Alpha IRR': -5.551115123125783e-17,
  'Sizing Alpha IRR': 0.0,
  'Residual Alpha IRR': 1.6653345369377348e-16})

In [17]:
number_of_simulations = 80
fund_size = 200
moic_results = []
irr_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    moic_results.append(model[0])
    irr_results.append(model[1])

In [13]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [18]:
avg_moic_decomposition = average_dicts(moic_results)
avg_irr_decomposition = average_dicts(irr_results)

In [19]:
avg_irr_decomposition

{'Portfolio IRR': 0.1254249410307245,
 'Market IRR': 0.12621241937725372,
 'Timing Alpha IRR': 0.0005482959373947698,
 'Strategy Alpha IRR': -0.0005876001447352805,
 'Geography Alpha IRR': -0.0006390110846440707,
 'Sizing Alpha IRR': -0.0002812446548742277,
 'Residual Alpha IRR': 0.00017208160032945938}

In [20]:
avg_moic_decomposition

{'Portfolio MOIC': 2.070848246598204,
 'Market MOIC': 2.077048239875696,
 'Timing Alpha MOIC': 0.004220220497024763,
 'Strategy Alpha MOIC': -0.004605679624816053,
 'Geography Alpha MOIC': -0.006274867234084672,
 'Sizing Alpha MOIC': -0.0007387338994095438,
 'Residual Alpha MOIC': 0.0011990669837915607}

## Analyze the individual effects

### strategy effect

In [21]:
venture_ids = cashflows[cashflows['Strategy'] == 'Venture']['FundID'].unique()
buyout_ids = cashflows[cashflows['Strategy'] == 'Buyout']['FundID'].unique()
growth_ids = cashflows[cashflows['Strategy'] == 'Growth']['FundID'].unique()
venture_ids = np.append(venture_ids,(buyout_ids[0], growth_ids[0])) # we need at least one venture fund to make the model work
buyout_ids = np.append(buyout_ids,(venture_ids[0], growth_ids[0])) # we need at least one buyout fund to make the model work
growth_ids = np.append(growth_ids,(venture_ids[0], buyout_ids[0])) # we need at least one growth fund to make the model work

In [22]:
def portfolio_cashflows_by_id(cashflows, ids):

    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in ids}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(ids)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [23]:
port_cfs_venture = portfolio_cashflows_by_id(cashflows, venture_ids)
port_cfs_venture = port_cfs_venture.copy()
port_cfs_buyout = portfolio_cashflows_by_id(cashflows, buyout_ids)
port_cfs_buyout = port_cfs_buyout.copy()
port_cfs_growth = portfolio_cashflows_by_id(cashflows, growth_ids)
port_cfs_growth = port_cfs_growth.copy()

In [24]:
model_venture = model_builder(cashflows, port_cfs_venture)
model_venture

({'Portfolio MOIC': 2.477724739794438,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 0.002178936706001,
  'Strategy Alpha MOIC': 0.3915476363696153,
  'Geography Alpha MOIC': -0.007143808374270755,
  'Sizing Alpha MOIC': 0.009885042451185821,
  'Residual Alpha MOIC': 0.004208692766208166},
 {'Portfolio IRR': 0.16231080459026645,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': -0.00010037599544629461,
  'Strategy Alpha IRR': 0.035877626070516505,
  'Geography Alpha IRR': -0.001120519286912508,
  'Sizing Alpha IRR': 0.0003730682834650334,
  'Residual Alpha IRR': 0.0010685861413898545})

In [25]:
model_buyout = model_builder(cashflows, port_cfs_buyout)
model_buyout

({'Portfolio MOIC': 1.5661356575774634,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': -0.021153402003938115,
  'Strategy Alpha MOIC': -0.5122061096673649,
  'Geography Alpha MOIC': -0.002282650075741355,
  'Sizing Alpha MOIC': 0.008902420275634038,
  'Residual Alpha MOIC': 0.015827159173175342},
 {'Portfolio IRR': 0.07413130414006164,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': -0.002400706313831144,
  'Strategy Alpha IRR': -0.051570243234361615,
  'Geography Alpha IRR': 0.0006525486819405624,
  'Sizing Alpha IRR': 0.0009371072048114382,
  'Residual Alpha IRR': 0.00030017842424853725})

In [26]:
model_growth = model_builder(cashflows, port_cfs_growth)
model_growth

({'Portfolio MOIC': 2.2020417939822634,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 0.014322735362735095,
  'Strategy Alpha MOIC': 0.1084796250779867,
  'Geography Alpha MOIC': 0.0007888033921679138,
  'Sizing Alpha MOIC': 0.0012459031024114608,
  'Residual Alpha MOIC': 0.0001564871712638194},
 {'Portfolio IRR': 0.13788069167468528,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': 0.0022181337631158315,
  'Strategy Alpha IRR': 0.009648525729245905,
  'Geography Alpha IRR': 0.00040019307654282077,
  'Sizing Alpha IRR': -0.00020617298081437863,
  'Residual Alpha IRR': -0.00039240729065875835})

### Geo effect

In [27]:
america_ids = cashflows[cashflows['Geography'] == 'North America']['FundID'].unique()
eu_ids = cashflows[cashflows['Geography'] == 'Europe']['FundID'].unique()
asia_ids = cashflows[cashflows['Geography'] == 'Asia']['FundID'].unique()
america_ids = np.append(america_ids,(eu_ids[0], asia_ids[0])) # we need at least one venture fund to make the model work
eu_ids = np.append(eu_ids,(america_ids[0], asia_ids[0])) # we need at least one buyout fund to make the model work
asia_ids = np.append(asia_ids,(america_ids[0], eu_ids[0])) # we need at least one growth fund to make the model work

In [28]:
port_cfs_america = portfolio_cashflows_by_id(cashflows, america_ids)
port_cfs_america = port_cfs_america.copy()
port_cfs_eu = portfolio_cashflows_by_id(cashflows, eu_ids)
port_cfs_eu = port_cfs_eu.copy()
port_cfs_asia = portfolio_cashflows_by_id(cashflows, asia_ids)
port_cfs_asia = port_cfs_asia.copy()

In [29]:
model_america = model_builder(cashflows, port_cfs_america)
model_america

({'Portfolio MOIC': 2.4487272534546416,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 0.008837681607898329,
  'Strategy Alpha MOIC': 0.005686414369052706,
  'Geography Alpha MOIC': 0.361763656830715,
  'Sizing Alpha MOIC': 0.005922772680208688,
  'Residual Alpha MOIC': -0.010531511908931535},
 {'Portfolio IRR': 0.15271255804549377,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': 0.0013241900981820764,
  'Strategy Alpha IRR': -0.0003692883251428525,
  'Geography Alpha IRR': 0.026616857119419385,
  'Sizing Alpha IRR': -0.00021534307182866086,
  'Residual Alpha IRR': -0.0008562771523900337})

In [30]:
model_eu = model_builder(cashflows, port_cfs_eu)
model_eu

({'Portfolio MOIC': 2.191249606111917,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': -0.005999860889433606,
  'Strategy Alpha MOIC': -0.016253504178979306,
  'Geography Alpha MOIC': 0.13750702438875084,
  'Sizing Alpha MOIC': -0.02546037396398626,
  'Residual Alpha MOIC': 0.02440808087986701},
 {'Portfolio IRR': 0.13652346773333268,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': -0.0007825587594537387,
  'Strategy Alpha IRR': -0.0015987591939207318,
  'Geography Alpha IRR': 0.013273669911768454,
  'Sizing Alpha IRR': -0.0026567836225028507,
  'Residual Alpha IRR': 0.0020754800201876894})

In [31]:
model_asia = model_builder(cashflows, port_cfs_asia)
model_asia

({'Portfolio MOIC': 1.6141299308582724,
  'Market MOIC': 2.0770482398756984,
  'Timing Alpha MOIC': 0.0001410064250326748,
  'Strategy Alpha MOIC': 0.0021234869047193072,
  'Geography Alpha MOIC': -0.48515998796289406,
  'Sizing Alpha MOIC': 0.011333315033178915,
  'Residual Alpha MOIC': 0.008643870582537128},
 {'Portfolio IRR': 0.08303605158675499,
  'Market IRR': 0.12621241937725386,
  'Timing Alpha IRR': -0.00010396549406457845,
  'Strategy Alpha IRR': 0.0018193088511911648,
  'Geography Alpha IRR': -0.04640103887403764,
  'Sizing Alpha IRR': 0.002137929609628525,
  'Residual Alpha IRR': -0.0006286018832163398})

### Fund quality

In [36]:
good_ids = cashflows[cashflows['FundQuality'] == 'Good']['FundID'].unique()
bad_ids = cashflows[cashflows['FundQuality'] == 'Bad']['FundID'].unique()
good_ids = np.append(good_ids,bad_ids[0]) # we need at least one venture fund to make the model work
bad_ids = np.append(bad_ids,good_ids[0]) # we need at least one buyout fund to make the model work

In [37]:
port_cfs_good = portfolio_cashflows_by_id(cashflows, good_ids)
port_cfs_good = port_cfs_good.copy()    
port_cfs_bad = portfolio_cashflows_by_id(cashflows, bad_ids)
port_cfs_bad = port_cfs_bad.copy()

In [38]:
model_good = model_builder(cashflows, port_cfs_good)
model_good

({'Portfolio MOIC': 2.963707594067612,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.0037403174076477974,
  'Strategy Alpha MOIC': 0.012488456861852626,
  'Geography Alpha MOIC': 0.0120526474932241,
  'Sizing Alpha MOIC': 0.0015763515336675304,
  'Residual Alpha MOIC': 0.6953874135358786},
 {'Portfolio IRR': 0.1981588060277388,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 0.0007184733930143683,
  'Strategy Alpha IRR': 0.0010990495998028371,
  'Geography Alpha IRR': 0.0005558753593686294,
  'Sizing Alpha IRR': -3.850548064013304e-05,
  'Residual Alpha IRR': 0.05493473783433911})

In [39]:
model_bad = model_builder(cashflows, port_cfs_bad)
model_bad

({'Portfolio MOIC': 1.541586884450072,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.0056064721734148115,
  'Strategy Alpha MOIC': -0.007845604096410774,
  'Geography Alpha MOIC': -0.0035993949528125846,
  'Sizing Alpha MOIC': 0.003069089554867288,
  'Residual Alpha MOIC': -0.6828931411174983},
 {'Portfolio IRR': 0.07082198288690467,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.0005822716148550233,
  'Strategy Alpha IRR': -0.0005440242572734566,
  'Geography Alpha IRR': 0.00045248933609678965,
  'Sizing Alpha IRR': 0.00045663114362046175,
  'Residual Alpha IRR': -0.0698500170425381})

### Vintage effect

In [40]:
vintage_ids = {}
for vint in cashflows['VintageYear'].unique():
    vint_ids = cashflows[cashflows['VintageYear'] == vint]['FundID'].unique()
    vintage_ids[vint] = vint_ids

In [41]:
import random
def pick_one_per_vintage(fund_dict):
    selected_ids = []

    for vintage, fund_ids in fund_dict.items():
        # Randomly pick one fund from this vintage
        fund_ids = list(fund_ids)  # ensure it's a list
        selected_id = random.choice(fund_ids)
        selected_ids.append(selected_id)

    return np.array(selected_ids)

one_from_each = pick_one_per_vintage(vintage_ids)

In [42]:
good_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 1]['FundID'].unique()
bad_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 0]['FundID'].unique()
good_vintage_ids = np.append(good_vintage_ids,one_from_each) # we need at least one venture fund to make the model work
bad_vintage_ids = np.append(bad_vintage_ids,one_from_each) # we need at least one buyout fund to make the model work

In [43]:
port_cfs_good_vintage = portfolio_cashflows_by_id(cashflows, good_vintage_ids)
port_cfs_good_vintage = port_cfs_good_vintage.copy()
port_cfs_bad_vintage = portfolio_cashflows_by_id(cashflows, bad_vintage_ids)
port_cfs_bad_vintage = port_cfs_bad_vintage.copy()

In [44]:
model_good_vintage = model_builder(cashflows, port_cfs_good_vintage)
model_good_vintage

({'Portfolio MOIC': 2.5391525268540347,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.3101218181352121,
  'Strategy Alpha MOIC': 0.0005587543279950147,
  'Geography Alpha MOIC': -0.010671429353007689,
  'Sizing Alpha MOIC': -0.008787322907759965,
  'Residual Alpha MOIC': 0.009468299416254045},
 {'Portfolio IRR': 0.16718252643789477,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 0.026856832157967442,
  'Strategy Alpha IRR': -0.0028248882741917503,
  'Geography Alpha IRR': -0.0039821055926791415,
  'Sizing Alpha IRR': -0.0005004487328571783,
  'Residual Alpha IRR': 0.006743961557801409})

In [45]:
model_bad_vintage = model_builder(cashflows, port_cfs_bad_vintage)
model_bad_vintage

({'Portfolio MOIC': 1.912453178799579,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.3269468866019516,
  'Strategy Alpha MOIC': -0.040376136416460096,
  'Geography Alpha MOIC': 0.052970839432698646,
  'Sizing Alpha MOIC': 0.0001589659647140973,
  'Residual Alpha MOIC': -0.011816010814763134},
 {'Portfolio IRR': 0.11190288665763065,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.028555901752591054,
  'Strategy Alpha IRR': -0.0071366844418671305,
  'Geography Alpha IRR': 0.0035140878355821026,
  'Sizing Alpha IRR': -0.0004233186167931918,
  'Residual Alpha IRR': 0.0036155283114459313})